In [1]:
import pandas as pd
import os
import numpy as np
import glob
import pyarrow.feather as feather
from pathlib import Path
pd.set_option('mode.chained_assignment', None)
def index_reset(df):
    return pd.RangeIndex(len(df.index))


Load df

In [2]:
# get your working directory and target folder that contains all your files
for nhan in ['VoIP','FileTransfer','Music','Youtube']:
    print(nhan)
    path = os.path.join(os.getcwd(), '/home/onos/FL/Data_Processing/send/rawds-20/gquic/' + nhan)
    files = [os.path.join(path, i) for i in os.listdir(
        path) if os.path.isfile(os.path.join(path, i))]
    summary_path = os.path.join(os.getcwd(), '/home/onos/FL/Data_Processing/Summary/' + nhan)
    sumary_files = [os.path.join(summary_path, i) for i in os.listdir(
        summary_path) if os.path.isfile(os.path.join(summary_path, i))]
    names = []
    for file in sumary_files:
        names.append((file.split('/'))[-1])
    for name in names:
        sum_path = summary_path + "/" + name
        pfile_name = name.replace('_summary','')
        pfile_path = path + "/" + pfile_name
        df_sum = pd.read_csv(sum_path)
        df = pd.read_csv(pfile_path, engine='pyarrow')
        df_stream_id = df[['stream_id', 'ip_src', 'port_src', 'ip_dst', 'port_dst']].groupby(['stream_id']).head(1)
        df_sum_final = pd.DataFrame()
        for _, row in df_stream_id.iterrows():
            result = df_sum.loc[ (df_sum['A_ip'] == row['ip_src']) & (df_sum['A_port'] == row['port_src']) &
                                (df_sum['B_ip'] == row['ip_dst']) & (df_sum['B_port'] == row['port_dst']) |
                                (df_sum['B_ip'] == row['ip_src']) & (df_sum['B_port'] == row['port_src']) & 
                                (df_sum['A_ip'] == row['ip_dst']) & (df_sum['A_port'] == row['port_dst']) ]
            result['stream_id'] = row['stream_id']
            df_sum_final = pd.concat([df_sum_final,result])
            del result
        final_path = summary_path.replace('Summary/','Summary/Final/')
        if not Path(final_path).exists():
            Path(final_path).mkdir(parents=True)
        df_sum_final.index = index_reset(df_sum_final)
        df_sum_final.to_feather(path=final_path+"/"+name.replace('.csv','.feather'))
        del df_sum,df_sum_final,df

VoIP
FileTransfer
Music
Youtube
